In [1]:
using Gen
using Serialization
using Logging
using BenchmarkTools

In [27]:
using GenArrow

In [3]:
function write_to_file(fname, input)
    seekstart(input)
    data = read(input, String)
    open(fname, "w") do io
        write(io, data)
    end
end
function read_from_file(fname)
    io = IOBuffer()
    open(fname, "r") do file
        data = read(file, String)
        write(io, data)
    end
    seekstart(io)
    io
end

read_from_file (generic function with 1 method)

### Leaf Nodes

In [23]:
@gen function submodel(w)
    a ~ bernoulli(0.5)
    b ~ bernoulli(0.5)
end

@gen function model(n)
    z ~ bernoulli(0.5)
    # for k=1:n
    #     @trace(bernoulli(0.5), k)
    # end
    q ~ submodel("what the")
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##model#316", Bool[0], false)

In [28]:
debugIO = open("write.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
tr_old, w_old = generate(model, (10,))
display(get_choices(tr_old))
display(get_score(tr_old))
io = IOBuffer()
GenArrow.serialize(io, tr_old)
close(debugIO)
write_to_file("data.trace", io)

│
├── :z : false
│
└── :q
    │
    ├── :a : true
    │
    └── :b : true


-2.0794415416798357

538

In [32]:
debugIO = open("read.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)

io = read_from_file("data.trace")
recovered_tr = try
    recovered_tr = GenArrow._deserialize(io, true)
    close(debugIO)
    recovered_tr
catch
    close(debugIO)
    rethrow()
end
# recovered_tr.trie
# display(get_score(recovered_tr))
# display(get_choices(recovered_tr))
# display(get_args(recovered_tr))

LazyTrace(Ptr: -1, Length: -1
[L]
 ┣━━ z ━ Gen.ChoiceOrCallRecord{Bool}(false, -0.6931471805599453, NaN, true)
 ┣━━ q ━ Gen.ChoiceOrCallRecord{LazyTrace}(LazyTrace(Ptr: -1, Length: -1
[L]
 ┣━━ a ━ Gen.ChoiceOrCallRecord{Bool}(true, -0.6931471805599453, NaN, true)
 ┣━━ b ━ Gen.ChoiceOrCallRecord{Bool}(true, -0.6931471805599453, NaN, true)
[I]
, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=538, maxsize=Inf, ptr=226, mark=-1), false, -2.772588722239781, 0.0, ("what the",), true), -2.772588722239781, 0.0, false)
[I]
, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=538, maxsize=Inf, ptr=226, mark=-1), false, -5.545177444479562, 0.0, (10,), true)

### Internal Nodes

In [33]:
@gen function model()
    {:x=>1} ~ bernoulli(0.5)
    {:x=>2} ~ categorical([0.25, 0.25, 0.25, 0.25])
    return 1
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[], false, Union{Nothing, Some{Any}}[], var"##model#317", Bool[], false)

In [34]:
debugIO = open("write.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
tr_old, w_old = generate(model, ())
display(get_choices(tr_old))
display(get_score(tr_old))
io = IOBuffer()
GenArrow.serialize(io, tr_old)
close(debugIO)
write_to_file("data.trace", io)

│
└── :x
    │
    ├── 2 : 2
    │
    └── 1 : false


-2.0794415416798357

332

In [35]:
debugIO = open("read.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
seekstart(io)
recovered_tr = try
    recovered_tr = GenArrow._deserialize(model, io, true)
    close(debugIO)
    recovered_tr
catch
    close(debugIO)
    rethrow()
end

LazyTrace(Ptr: -1, Length: -1
[L]
[I]
 ┣━━ x ━┓
 ┃      Ptr: 197, Length: 135
 ┃      [L]
 ┃       ┣━━ 2 ━ Gen.ChoiceOrCallRecord{Int64}(2, -1.3862943611198906, NaN, true)
 ┃       ┣━━ 1 ━ Gen.ChoiceOrCallRecord{Bool}(false, -0.6931471805599453, NaN, true)
 ┃      [I]
, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=332, maxsize=Inf, ptr=197, mark=-1), false, -4.1588830833596715, 0.0, (), 1)